In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, Reshape, Conv1D, UpSampling1D, Flatten, Dropout, Input
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

In [44]:
#1. data min max scaler
#2. add embedding for amount feature
#...
class AnoOCGAN:
    
    def __init__(self, noise_size):
        self.noise_size = noise_size
        self.losses = 'binary_crossentropy'
        self.opt = 'adam'
        self.build_gan_model()
        
        self.x_size = 896
        self.h_size = 200
        self.y_size = 1
        
        self.theta = np.random.normal(0, 1, self.h_size + self.h_size*self.x_size + 1)
        #rvalue = np.random.normal(0, 1, (len(X_normal), y_size))
        self.nu = 0.04
        self.g = lambda x: 1 / (1 + tf.exp(-x)) 
        self.rvalue = 0.1
        
    def build_g(self):
        
        g_model = Sequential()
        g_model.add(Dense(896,input_shape = (100,) , kernel_initializer=tf.keras.initializers.glorot_normal()))
        g_model.add(LeakyReLU(0.2))
        g_model.add(Reshape((7,128)))
        g_model.add(UpSampling1D(size=2))
        g_model.add(Conv1D(64,5, padding="same"))
        g_model.add(LeakyReLU(0.2))
        g_model.add(UpSampling1D(size=2))
        g_model.add(Conv1D(1, 5, padding="same", activation = 'tanh'))
        g_model.add(Flatten())
        
        g_model.compile(loss=self.losses, optimizer=self.opt)

        return g_model
    
    def build_d(self):
        
        d_model = Sequential()
        d_model.add(Reshape((28,1), input_shape=(28,)))
        d_model.add(Conv1D(64,5, strides=2, padding="same"))
        d_model.add(LeakyReLU(0.2))
        d_model.add(Dropout(0.3))
        d_model.add(Conv1D(128, 5, strides=2, padding="same"))
        d_model.add(LeakyReLU(0.2))
        d_model.add(Dropout(0.3))
        d_model.add(Flatten())
        d_model.add(Dense(1, activation="sigmoid"))
        
        d_model.compile(loss = self.losses, optimizer=self.opt)

        return d_model
    
    def build_g_and_d(self):
        
        gan_input = Input(shape=(self.noise_size,), name="g_d_input")
        img = self.g_model(gan_input)
        g_out = self.d_model(img)
        
        gan = Model(inputs = gan_input, outputs = g_out)
        gan.compile(loss=self.losses, optimizer=self.opt)
        
        return gan
        
    def build_gan_model(self):

        self.g_model = self.build_g()
        self.d_model = self.build_d()
        self.g_d_model = self.build_g_and_d()
        
    def train_gan_model(self, X_train, BATCH_SIZE):
        #d_g = self.build_d_and_g()
        #d = tf.keras.Model(inputs = d_g.layers[0].layers[0], ouputs = d_g.layers[0].layers[-1])
        for epoch in tqdm(range(100)):
            for bat in range(50):
                #---------------------train d model---------------------
                self.d_model.trainable = True

                idx = np.random.randint(0, X_train.shape[0], BATCH_SIZE)
                noise = np.random.normal(0, 1, (BATCH_SIZE, self.noise_size))
                real_X = X_train[idx]
                fake_X = self.g_model(noise)
                #print(self.d_model(real_X))
                #self.d_model.train_on_batch(fake_X, np.array([0] * BATCH_SIZE))
                d_loss_fake = self.d_model.train_on_batch(fake_X, np.array([0] * BATCH_SIZE))
                d_loss_real = self.d_model.train_on_batch(real_X, np.array([1] * BATCH_SIZE))
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                #---------------------train g and d model---------------------
                self.d_model.trainable = False
                noise = np.random.normal(0, 1, (BATCH_SIZE, self.noise_size))
                g_d_loss = self.g_d_model.train_on_batch(noise, np.array([1] * BATCH_SIZE))

            print("d_loss_f : %s, d_loss_r : %s, g_d_loss : %s"%(d_loss_fake,d_loss_real ,g_d_loss))
            
            self.d_model.save_weights('assets/discriminator', True)
            self.g_model.save_weights('assets/generator', True)
        
    def load_models(self):
        self.d_model.load_weights('assets/discriminator')
        self.g_model.load_weights('assets/generator')
        
    def generate(self, BATCH_SIZE):
        return self.g_model.predict(np.random.normal(0, 1, (BATCH_SIZE, self.noise_size)))
    
    def build_d_feature_model(self):
        self.d_model = self.build_d()
        self.d_feature_model = Model(inputs = self.d_model.layers[0].input, outputs = self.d_model.layers[-5].output)
        self.d_feature_model.compile(loss = self.losses, optimizer=self.opt)
        
    def build_ano_model(self):
        initial = RandomNormal(mean=0, stddev=0.00001)
        self.load_models()
        self.g_d_model = self.build_g_and_d()
        self.build_d_feature_model()
        #self.g_model.trainable = False
        #self.d_feature_model.trainable = False
        
        self.W1 = tf.Variable(initial_value=initial((self.x_size,self.h_size), dtype=tf.float64), dtype=tf.float64)
        self.W2 = tf.Variable(initial_value=initial((self.h_size,self.y_size), dtype=tf.float64), dtype=tf.float64)
        self.trainable = [self.W1, self.W2]
        
    def feed_forward(self,X):
        h = tf.nn.sigmoid(tf.matmul(X, self.W1))
        yhat = tf.matmul(h, self.W2)
        return yhat
        
    def OC_cost(self, X, r):
        X = tf.cast(X, tf.float32)
        W = tf.cast(self.W1, tf.float32)
        V = tf.cast(self.W2, tf.float32)

        term1 = 0.5 * tf.reduce_sum(W**2)
        term2 = 0.5 * tf.reduce_sum(V**2)
        term3 = 1/nu * tf.reduce_mean(tf.nn.relu(r - nnScore(X, W, V, self.g)))
        term4 = -r
        return term1 + term2 + term3 + term4
    
    def nnScore(self, X, w, V):
        X = tf.cast(X, tf.float32)
        w = tf.cast(w, tf.float32)
        V = tf.cast(V, tf.float32)
        
        return tf.matmul(self.g((tf.matmul(X, w))), V)

    def train_ano_model(self, X_normal, epoch):
        
        self.build_ano_model()
        optimizer = SGD(0.0001)
        #self.rvalue = 0.1
        
        for epoch in tqdm(range(epoch)):

            with tf.GradientTape() as tape:
                #yhat = forwardprop(X_normal,W1,W2)
                #predict = tf.argmax(yhat, axis=1)
                cost = self.OC_cost(X_normal, self.rvalue)
                grad_value = tape.gradient(cost, self.trainable)
                #rvalue = rvalue.eval()
                
            optimizer.apply_gradients(zip(grad_value, self.trainable))
            self.rvalue = self.nnScore(X_normal, self.W1, self.W2, self.g)

            self.rvalue = np.percentile(self.rvalue, q=100*self.nu)

            if epoch % 50 == 0:
                print("Epoch = %d, r = %f" % (epoch+1, self.rvalue))
                
    #def check_trained_gan(self):
   

In [6]:
data = pd.read_csv("data/creditcard.csv")
data = data.drop(["Time", "Amount"], axis=1)

In [7]:
scaler = MinMaxScaler()
scaler.fit(data.drop("Class", axis=1))

MinMaxScaler(copy=True, feature_range=(0, 1))

In [8]:
data_normal = data[data["Class"]==0]
data_normal = data_normal.drop("Class", axis=1)
#data_normal = scaler.transform(data_normal)

In [9]:
data_normal.shape

(284315, 28)

In [21]:
data_anormal = data[data["Class"]==1]
data_anormal = data_anormal.drop("Class", axis=1)
#data_anormal = scaler.transform(data_anormal)

In [45]:
a = AnoOCGAN(100)

In [42]:
a.d_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_7 (Reshape)          (None, 28, 1)             0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 14, 64)            384       
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 14, 64)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 14, 64)            0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 7, 128)            41088     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 7, 128)            0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 7, 128)           

In [46]:
a.train_gan_model(data_normal, 30)


  0%|          | 0/100 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




  1%|          | 1/100 [00:04<07:55,  4.80s/it]

d_loss : 0.39535054564476013, g_d_loss : 1.0233994



  2%|▏         | 2/100 [00:09<07:47,  4.77s/it]

...

d_loss : 7.666666507720947, g_d_loss : 0.0



 99%|█████████▉| 99/100 [07:52<00:04,  4.87s/it]

d_loss : 7.666621208190918, g_d_loss : 0.0



100%|██████████| 100/100 [07:57<00:00,  4.78s/it]

d_loss : 7.666623592376709, g_d_loss : 0.0


In [47]:
f = a.g_model.predict(np.random.normal(0, 1, (10, 100)))

In [48]:
a.d_model.predict([data_normal[1:10]])

array([[0.9999999 ],
       [0.9999999 ],
       [0.9999999 ],
       [1.        ],
       [0.9999999 ],
       [1.        ],
       [0.99999994],
       [1.        ],
       [1.        ]], dtype=float32)

In [49]:
a.d_model.predict([data_anormal[1:10]])

array([[0.9999999 ],
       [0.9999998 ],
       [0.99999964],
       [0.9999999 ],
       [0.9999999 ],
       [0.9999999 ],
       [0.9999998 ],
       [0.9999999 ],
       [0.9999999 ]], dtype=float32)

In [50]:
a.d_model.predict(f)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

In [161]:
d = a.d_model

In [162]:
d.train_on_batch(data_normal[1:3], np.array([1,1]))

[0.73150027, 0.0]

In [157]:
a = Sequential()
a.add(Reshape((28,1)))
a.add(Conv1D(64,5, strides=2, padding="same"))
a.add(LeakyReLU(0.2))
a.add(Dropout(0.3))
a.add(Conv1D(128, 5, strides=2, padding="same"))
a.add(LeakyReLU(0.2))
a.add(Dropout(0.3))
a.add(Flatten())
a.add(Dense(1, activation="sigmoid"))

In [158]:
a.compile(loss='binary_crossentropy', optimizer='sgd')

In [133]:
a.train_on_batch(data_normal[idx], np.array([1]*30))

0.66720396

In [166]:
a.train_gan_model(data_normal, 30)

  0%|          | 0/200 [00:00<?, ?it/s]


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



KeyboardInterrupt: 

In [58]:
np.array([[0]] * 30).shape

(30, 1)

In [21]:
a.g_d_model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_8 (Sequential)    (None, 28)                131841    
_________________________________________________________________
sequential_9 (Sequential)    (None, 1)                 42369     
Total params: 174,210
Trainable params: 174,210
Non-trainable params: 0
_________________________________________________________________


In [136]:
def build_g():
    g_model = Sequential()
    g_model.add(Dense(896,kernel_initializer=tf.keras.initializers.glorot_normal()))
    g_model.add(LeakyReLU(0.2))
    g_model.add(Reshape((7,128)))
    g_model.add(UpSampling1D(size=2))
    g_model.add(Conv1D(64,5, padding="same"))
    g_model.add(LeakyReLU(0.2))
    g_model.add(UpSampling1D(size=2))
    g_model.add(Conv1D(1, 5, padding="same"))
    g_model.add(Flatten())


    #inputs = tf.keras.Input(self.noise_size, name="g_input")
    #dense1 = tf.keras.layers.Dense(896,kernel_initializer=tf.keras.initializers.RandomNormal(stddev=0.02))(inputs)
    #lrelu1 = tf.keras.layers.LeakyReLU(0.2)(dense1)
    #reshape1 = tf.keras.layers.Reshape((7,128))(lrelu1)
    #upsampling1 = tf.keras.layers.UpSampling1D(size=2)(reshape1)
    #conv1 = tf.keras.layers.Conv1D(64,5, padding="same")(upsampling1)
    #lrelu2 = tf.keras.layers.LeakyReLU(0.2)(conv1)
    #upsampling2 = tf.keras.layers.UpSampling1D(size=2)(lrelu2)
    #conv2 = tf.keras.layers.Conv1D(1, 5, padding="same")(upsampling2)
    #out = tf.keras.layers.Flatten()(conv2)
    #model = tf.keras.Model(inputs = inputs, outputs = out)

    return g_model

def build_d():
    d_model = Sequential()
    d_model.add(Reshape((28,1)))
    d_model.add(Conv1D(64,5, strides=2, padding="same"))
    d_model.add(LeakyReLU(0.2))
    d_model.add(Dropout(0.3))
    d_model.add(Conv1D(128, 5, strides=2, padding="same"))
    d_model.add(LeakyReLU(0.2))
    d_model.add(Dropout(0.3))
    d_model.add(Flatten())
    d_model.add(Dense(1, activation="sigmoid"))


    #inputs = tf.keras.Input((28), name="d_input")
    #reshape1 = tf.keras.layers.Reshape((28,1))(inputs)
    #conv1 = tf.keras.layers.Conv1D(64,5, strides=2, padding="same")(reshape1)
    #lrelu1 = tf.keras.layers.LeakyReLU(0.2)(conv1)
    #drop1 = tf.keras.layers.Dropout(0.3)(lrelu1)
    #conv2 = tf.keras.layers.Conv1D(128, 5, strides=2, padding="same")(drop1)
    #lrelu2 = tf.keras.layers.LeakyReLU(0.2)(conv2)
    #drop2 = tf.keras.layers.Dropout(0.3)(lrelu2)
    #flat = tf.keras.layers.Flatten()(drop2)
    #out = tf.keras.layers.Dense(1)(flat)
    #model = tf.keras.Model(inputs = inputs, outputs = out)

    return d_model

def build_d_and_g():

    g_model = build_g()
    #g_model.compile(loss='binary_crossentropy', optimizer='adam')
    d_model = build_d()
    #d_model.compile(loss='binary_crossentropy', optimizer='adam')



    d_and_g_model = tf.keras.Sequential()
    d_and_g_model.add(g_model)
    d_and_g_model.add(d_model)


    return d_and_g_model

def train_gan_model():
    d_g = build_d_and_g()
    d = tf.keras.Model(inputs = d_g.layers[0].layers[0], ouputs = d_g.layers[0].layers[-1])


